# Chapter 12: Custom Models and Training with TensorFlow

In [1]:
# Preliminaries
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

# Random seeds
np.random.seed(42)
tf.random.set_seed(42)

# Plots
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

## 1. Overview of TensorFlow

__Summary__ :
  - Core is similar to numpy, but with GPU support
  - Supports distributed computing
  - JIT (just-in-time) compiler; extracts _computation graph_ from Python function and optimizes + runs
  - Can export computation graph
  - Autodiff, optimizer implementation
  - Each tf op implemented in C++ (can write own ops in C++ API)
  
__Other notable features__ :
  - `tf.keras`
  - Data loading and preprocessing: `tf.data`, `tf.io`, ...
  - Image processing: `tf.image`
  - Signal processing: `tf.signal`

### 1.1 Tensors and Operations

#### 1.1.1 Basics

Can create tensor with `tf.constant()`

In [2]:
# Example: tensor matrix with two rows, three cols of floats

tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]) # matrix!

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
# Scalar
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

Each `tf.Tensor` has shape and data type (`dtype`)

In [4]:
t = tf.constant([[1., 2., 3.], [4., 5., 6,]])
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

Similar indexing to numpy

In [6]:
t[:, 1:] # every row, col 2+

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[..., 1, tf.newaxis] # every row, only column 1

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

#### 1.1.2 Tensor operations

In [11]:
# Both equivalent forms of addition
t + 10
tf.add(t, 10)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [9]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [12]:
# For matrix multiplication - Python 3.5+
t @ tf.transpose(t)

# Alternatively
tf.matmul(t, tf.transpose(t))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

#### 1.1.3 Other considerations

When names differ from np implementation, typically have a reason!

e.g. `tf.transpose(t)` creates a new tensor w/copy of transposed data, vs np's `t.T`

#### 1.1.4 Keras low-level API

keras low-level backend: `keras.backend`
- In `tf.keras`, these call equivalent tf ops
- More functions available in tf directly

In [13]:
K = keras.backend
K.square(K.transpose(t)) + 10

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[11., 26.],
       [14., 35.],
       [19., 46.]], dtype=float32)>

#### 1.1.5 Tensors and NumPy

- Can create tensor from np array & vice-versa
- Can apply tf ops to np arrays & vice-versa

In [14]:
a = np.array([2., 4., 5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [15]:
# tensor to np array
t.numpy()

# Equivalently
np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [16]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [17]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

Main difference: np uses 64-bit, tf uses 32-bit

### 1.2 Type conversions

- Significantly hurt performance
- tf does not do these automatically; instead raises exception

In [18]:
# Can't add float and integer
try:
    tf.constant(2.0) + tf.constant(40)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]


In [19]:
# Can't add floats of different precision
try:
    tf.constant(2.0) + tf.constant(40.0, dtype = tf.float64)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]


In [20]:
# Can use tf.cast to convert types
t2 = tf.constant(40.0, dtype = tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>